In [62]:
import json
import urllib
import os
import requests
import json
import numpy as np
import pandas as pd

from collections import defaultdict, namedtuple
from tqdm import tqdm

In [63]:
def record_size_from_dir(dir):
    with open(dir + '/record_size', 'r') as fd:
        return int(fd.read().split('\t')[0])

def map_from_dir(dir):
    fn = dir + '/map.old_to_new.i'
    fn_len = os.path.getsize(fn)
    return np.memmap(fn, dtype=np.int32, shape=(int(fn_len/4)), mode='r')

def embedding_from_dir(dir, K):
    fn = dir + '/embedding.f'
    fn_len = os.path.getsize(fn)
    return np.memmap(fn, dtype=np.float32, shape=(int(fn_len/(4*K)), K), mode='r')

def directory_to_config(dir):
    K = record_size_from_dir(dir)
    return { 'record_size' : K,
             'dir' : dir,
             'map' : map_from_dir(dir),
             'embedding' : embedding_from_dir(dir, K)}

In [64]:
d = '/Volumes/JSALT-10T/JSALTdir/semantic_scholar/embeddings/proposed'

map0 = config['map']
emb0 = config['embedding']

In [65]:
def flatten(l):
    return [item for sublist in l for item in sublist]

# build a named_tuple of idx:[src, tgt]
def sem_scho_ids_to_dataset(src_to_tgt_sem_scho_ids):
    dataset = {}
    for i, (src, tgt) in enumerate(src_to_tgt_sem_scho_ids.items()):
        dataset[i] = sample(src, tgt)
    return(dataset)

In [66]:
# intermediate step for leave-one-out
paper_id = 16088675
leave_one_out = dict()
dummy = [17096321, 53531523, 4852360, 11330317, 13091213, 13029280, 3513255, 212734631, 84855733, 10084791, 21463620, 41764805, 20323655, 5733836, 26758480, 33932634, 201212387, 13683813, 7710565, 13677159, 30309488, 206501362, 4704627, 25214071, 20550907]
for d in dummy:
    l = dummy
    l.remove(d)
    leave_one_out[d] = l
    

In [67]:
ann_out = {}

# process one example
for k, v in leave_one_out.items():
    for idx in v:
        try:
            paper_emb = emb0[map0[idx]]
            arr = np.array(paper_emb)
            ann_out[(k, idx)] = arr
        except:
            print(f'failed with {idx}')

In [68]:
JSALTdir='/Volumes/JSALT-10T/JSALTdir'
!export JSALTdir=/Volumes/JSALT-10T/JSALTdir
!export JSALTsrc=/Users/melissacmitchell/Documents/research/JSALT/changes/JSALT_Better_Together/src
!export proposed=$JSALTdir/semantic_scholar/embeddings/proposed

In [69]:
query = np.mean(np.vstack(ann_out.values()), axis=0)
query.tofile('/tmp/vec')
print(query)

[-1.08762197e-02 -8.22888315e-02  5.57945147e-02  1.17461253e-02
 -9.85247120e-02  3.26371491e-02  1.42152086e-01  2.27540657e-02
 -9.18108597e-03 -7.41021484e-02  1.91149600e-02 -7.35447556e-02
  9.93810296e-02  4.10923362e-02 -2.35390216e-02  8.44905898e-03
  1.25471447e-02  1.93591684e-01 -1.46575866e-03 -1.27604860e-03
 -2.74218377e-02 -2.19069738e-02  2.53473204e-02  6.73216581e-02
  6.65730014e-02 -3.53074670e-02 -1.36303380e-02 -1.13746114e-01
  4.32060845e-03  3.46142240e-02 -9.31833014e-02  5.02614900e-02
 -1.89630032e-01  3.80791770e-03  6.37612641e-02  8.81241485e-02
  9.33601782e-02 -3.01767010e-02 -9.95438024e-02 -2.89862929e-03
  3.48748751e-02 -7.16617405e-02  1.37317032e-01  4.19444107e-02
  3.17958556e-02 -1.78702790e-02 -7.59725943e-02 -3.02933939e-02
  1.47152960e-01 -3.10279094e-02 -5.18208779e-02 -8.34400132e-02
 -8.77052546e-02 -6.40808512e-03  5.98752908e-02 -1.04949936e-01
 -3.97851132e-02  5.94490301e-03  1.34705016e-02 -7.03794369e-03
  3.51301283e-02  4.62906

/var/folders/3g/5sw9jzy51nzcdg131bwkqh2w0000gp/T/ipykernel_54042/1324754142.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  query = np.mean(np.vstack(ann_out.values()), axis=0)
